In [10]:
#Importing all the required packages
import os
import numpy as np
from skimage import io, color, feature, transform
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression


In [11]:
# Paths to the images
cancer_folder = "C:/Users/SERVER/Desktop/ImageClassification/Lung_datasSet/lung_aca"
no_cancer_folder = "C:/Users/SERVER/Desktop/ImageClassification/Lung_datasSet/lung_n"
cancer_scc_folder = "C:/Users/SERVER/Desktop/ImageClassification/Lung_datasSet/lung_scc"

#File type accesptable
valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']


In [12]:
cancer_images =[io.imread(os.path.join(cancer_folder, image_path))
               for image_path in os.listdir(cancer_folder)
               if os.path.splitext(image_path)[1].lower() in valid_extensions]

no_cancer_images =[io.imread(os.path.join(no_cancer_folder, image_path))
               for image_path in os.listdir(no_cancer_folder)
               if os.path.splitext(image_path)[1].lower() in valid_extensions]

cancer_scc_images =[io.imread(os.path.join(cancer_scc_folder, image_path))
               for image_path in os.listdir(cancer_scc_folder)
               if os.path.splitext(image_path)[1].lower() in valid_extensions]

In [13]:
#hog feature extraction function
def extract_features(image):
    image_resized = transform.resize(image, (128,128))
    gray = color.rgb2gray(image_resized)
    hog_features = feature.hog(gray, pixels_per_cell =(8,8), cells_per_block=(2,2))
    return hog_features

In [14]:
#feature extraction using GLCM
def extract_glcm_features(image):
    if len(image.shape) == 3:
        gray = color.rgb2gray(image)
    else:
        gray = image
    image_8bit = (gray*255).astype(np.uint8)
    glcm = graycomatrix(image_8bit, [1], [0], 256, symmetric = True, normed = True)
    contrast = graycoprops(glcm, 'contrast')[0,0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0,0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0,0]
    energy = graycoprops(glcm, 'energy')[0,0]
    correlation = graycoprops(glcm, 'correlation')[0,0]
    return [contrast, dissimilarity, homogeneity, energy, correlation]

In [15]:
#feature extraction using LBP
def extract_lbp_features(image):
    gray = color.rgb2gray(image)
    lbp = local_binary_pattern(gray, P = 8, R = 1, method = 'uniform')
    lbp_hist, _=np.histogram(lbp, bins = np.arange(0, lbp.max() + 1))
    lbp_hist = lbp_hist.astype(float)
    lbp_hist /=(lbp_hist.sum() + 1e-6)
    return lbp_hist

In [16]:
#extracting color using color histogram
def extract_color_histogram(image, bins = 256):
    if len(image.shape) == 2:
        hist = np.histogram(image, bins = bins, range=(0,256))[0]
    elif len(image.shape) == 3 and image.shape[2] == 3:
        hist_r = np.histogram(image[:,:,0], bins=bins, range=(0,256))[0]
        hist_g = np.histogram(image[:,:,0], bins=bins, range=(0,256))[0]
        hist_b = np.histogram(image[:,:,0], bins=bins, range=(0,256))[0]
        hist = np.concatenate((hist_r,hist_g,hist_b))
    else:
        raise ValueError("Unsupport image format")
    return hist
        

In [17]:
# function to combine both hog and glcm and lbp feature functions
def combined_features(image):
    hog_feat = extract_features(image)
    glcm_feat = extract_glcm_features(image)
    lbp_feat = extract_lbp_features(image)
    color_hist_feat = extract_color_histogram(image)
    return np.concatenate([hog_feat, glcm_feat, lbp_feat, color_hist_feat])

In [18]:
cancer_combined_features = np.array([combined_features(img) for img in cancer_images])
no_cancer_combined_features = np.array([combined_features(img) for img in no_cancer_images])
cancer_scc_combined_features = np.array([combined_features(img) for img in cancer_scc_images])

In [19]:
# Data preparation
X = np.vstack((cancer_combined_features, no_cancer_combined_features, cancer_scc_combined_features))

y = np.hstack((
    np.ones(cancer_combined_features.shape[0]),        # Label 1 for lung adenocarcinoma (ACA)
    -1 * np.ones(no_cancer_combined_features.shape[0]), # Label -1 for normal lung tissue
    2 * np.ones(cancer_scc_combined_features.shape[0])  # Label 2 for lung squamous cell carcinoma (SCC)
))


In [20]:
#Splitting the data into and train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0) 

In [21]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel = 'linear', C = 1, gamma = 'scale')
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 83.33%


In [22]:
# function to predict the validation images
def predict_image(image_path, classifier, feature_extractor):
    img = io.imread(image_path)
    features = feature_extractor(img)
    prediction = classifier.predict([features])
    return prediction[0]

In [24]:
# looking the images from the validation data
check_folder = "C:/Users/SERVER/Desktop/ImageClassification/Lung_datasSet/check"
for image_name in os.listdir(check_folder):
    if os.path.splitext(image_name)[1].lower() in valid_extensions:
        image_path = os.path.join(check_folder,image_name)
        predicted_class = predict_image(image_path, svm_classifier, combined_features)
        
        if predicted_class == 1:
            print(f"The image {image_name} is cancerous.")
        elif predicted_class == 2:
            print(f"The image {image_name} is scc cancerous.")
        else:
            print(f"The image {image_name} is non cancerous.")
            


C:\Users\SERVER\anaconda3\Lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


The image lungaca1.jpeg is cancerous.


C:\Users\SERVER\anaconda3\Lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


The image lungaca2.jpeg is cancerous.


C:\Users\SERVER\anaconda3\Lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


...

The image lungscc3.jpeg is scc cancerous.


C:\Users\SERVER\anaconda3\Lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


The image lungscc4.jpeg is scc cancerous.


C:\Users\SERVER\anaconda3\Lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


The image lungscc5.jpeg is scc cancerous.


In [25]:
# splitting the data into k folds and then feeding them to the model to avoid over fitting
from sklearn.model_selection import StratifiedKFold
k = 5
skf = StratifiedKFold(n_splits = k, shuffle = True, random_state = 5)

accuracies = []

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    svm_classifier = SVC(kernel = 'linear', C = 1, gamma = 'scale')
    svm_classifier.fit(X_train, y_train)
    y_pred = svm_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f'Accuracy: {accuracy * 100:.2f}%')

    

Accuracy: 66.67%
Accuracy: 70.00%
Accuracy: 66.67%
Accuracy: 78.33%
Accuracy: 71.67%


In [26]:
average_accuracy = np.mean(accuracies)
print(f'Average accuracy over {k} folds:{average_accuracy * 100:.2f}%') 

Average accuracy over 5 folds:70.67%
